# Profile Randomization Demo
This notebook demonstrates how to use the profile randomization system to create diverse behavioral agent combinations and run comparative experiments.


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from profiles_beergame.engine.simulation import SimulationRunner, SimulationConfig, constant_demand
from profiles_beergame.agents.profile_randomizer import ProfileRandomizer, create_random_agents, create_mixed_scenario, get_available_profiles
from profiles_beergame.metrics.analytics import compute_bullwhip, summarize_kpis
from profiles_beergame.plots.plotting import plot_time_series


## 1. Basic Profile Randomization


In [ ]:
# Show available profiles
print("Available behavioral profiles:")
for profile in get_available_profiles():
    print(f"  - {profile.replace('_', ' ').title()}")

# Create a completely random scenario
randomizer = ProfileRandomizer(seed=42)
scenario = randomizer.create_random_scenario()

print("\nRandom Scenario:")
print(randomizer.describe_scenario(scenario))


In [ ]:
# Run simulation with random agents
agents = randomizer.create_agents_dict(scenario)
config = SimulationConfig(periods=50, random_seed=42)
runner = SimulationRunner(agents, constant_demand(4), config)
results = runner.run()

print("Simulation completed!")
print(f"Generated {len(results)} data points")
print("\nFirst few rows:")
results.head()


## 2. Compare Different Profile Combinations


In [ ]:
# Create specific combinations for comparison
print("Creating mixed scenarios with specific profiles...\n")

# Scenario 1: Aggressive retailer with conservative upstream
agents_1 = create_mixed_scenario(
    retailer_profile="aggressive_growth_hacker",
    wholesaler_profile="conservative_custodian",
    distributor_profile="conservative_custodian",
    factory_profile="passive_pipeline",
    seed=1
)
print("Scenario 1: Aggressive retailer with conservative upstream")

# Scenario 2: Signal chasing throughout the chain
agents_2 = create_mixed_scenario(
    retailer_profile="signal_chaser",
    wholesaler_profile="signal_chaser",
    distributor_profile="myopic_firefighter",
    factory_profile="aggressive_growth_hacker",
    seed=2
)
print("Scenario 2: Signal chasers with reactive factory")

# Scenario 3: All random baseline for comparison
agents_3 = create_mixed_scenario(
    retailer_profile="random_baseline",
    wholesaler_profile="random_baseline",
    distributor_profile="random_baseline",
    factory_profile="random_baseline",
    seed=3
)
print("Scenario 3: All random baseline (control)")


In [ ]:
# Run all three scenarios and compare
scenarios_to_test = [
    ("Aggressive + Conservative", agents_1),
    ("Signal Chasers + Reactive", agents_2),
    ("Random Baseline", agents_3)
]

all_results = []
bullwhip_comparison = []

for scenario_name, agents in scenarios_to_test:
    # Run simulation
    config = SimulationConfig(periods=50, random_seed=42)
    runner = SimulationRunner(agents, constant_demand(4), config)
    results = runner.run()
    results['scenario'] = scenario_name
    all_results.append(results)
    
    # Calculate bullwhip
    bullwhip = compute_bullwhip(results)
    bullwhip['scenario'] = scenario_name
    bullwhip_comparison.append(bullwhip)
    
    print(f"\n{scenario_name} - Bullwhip Factors:")
    print(bullwhip[['role', 'bullwhip_factor']].round(3))

# Combine all results
combined_results = pd.concat(all_results, ignore_index=True)
combined_bullwhip = pd.concat(bullwhip_comparison, ignore_index=True)


In [ ]:
# Visualize bullwhip comparison
fig, ax = plt.subplots(1, 1, figsize=(12, 6))

# Create grouped bar chart
scenarios = combined_bullwhip['scenario'].unique()
roles = ['retailer', 'wholesaler', 'distributor', 'factory']
x = range(len(roles))
width = 0.25

for i, scenario in enumerate(scenarios):
    scenario_data = combined_bullwhip[combined_bullwhip['scenario'] == scenario]
    values = [scenario_data[scenario_data['role'] == role]['bullwhip_factor'].iloc[0] 
              for role in roles]
    ax.bar([xi + width*i for xi in x], values, width, label=scenario, alpha=0.8)

ax.set_xlabel('Supply Chain Role')
ax.set_ylabel('Bullwhip Factor')
ax.set_title('Bullwhip Effect Comparison Across Scenarios')
ax.set_xticks([xi + width for xi in x])
ax.set_xticklabels([role.capitalize() for role in roles])
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()


## 3. Time Series Visualization


In [ ]:
# Plot time series for the most interesting scenario
most_volatile_scenario = "Signal Chasers + Reactive"
scenario_data = combined_results[combined_results['scenario'] == most_volatile_scenario]

print(f"Time series for: {most_volatile_scenario}")
plot_time_series(scenario_data)


## Summary

This notebook demonstrated how to:

1. **Create random agent combinations** using the ProfileRandomizer
2. **Compare different behavioral profiles** across supply chain roles
3. **Analyze bullwhip effects** resulting from different agent combinations
4. **Visualize results** to understand supply chain dynamics

The profile randomization system makes it easy to explore how different decision-making patterns interact to create complex supply chain behaviors. Try experimenting with different profile combinations to see how they affect the bullwhip effect!
